# Tutorial4: estimate 3D emission from EHT observations

---
This tutorial demonstrates the recovery of 3D emission from synthetic observations. Complex visibilities or closure phases are used to fit the parameters of a coordinate-based neural network (NeRF).

In [1]:
import bhnerf
import kgeo
import ehtim as eh
from ehtim.observing.obs_helpers import ftmatrix
import ehtim.const_def as ehc
import bhnerf.constants as consts
from astropy import units
import jax

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import os
from datetime import datetime
from tqdm.notebook import tqdm

# Runing on 2 GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-zrmemqxx because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2022-09-29 17:21:27.404483: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [2]:
"""
Generate synthetic observations of a hot-spot
"""
spin = 0.2
inclination = np.deg2rad(60.0)

array = 'ngEHT'             
nt = 64                             # number of image frames
flux_scaling = 0.02                 # scale image-plane values to `reasonable` fluxes  
fov_M = 16.0                        # field of view (M)
tstart = 2.0 * units.hour           # observation start time
tstop = tstart + 40.0 * units.min   # observation stop time

# Compute geodesics (see Tutorial1)
geos = bhnerf.kgeo.image_plane_geos(
    spin, inclination, 
    num_alpha=64, num_beta=64, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2]
)
Omega = np.sign(spin + np.finfo(float).eps) * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
t_injection = -float(geos.r_o)

# Generate hotspot measurements (see Tutorial2) 
emission_0 = flux_scaling * bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=0.0,
    orbit_radius=5.5,
    std=0.7,
    r_isco=bhnerf.constants.isco_pro(spin),
    fov=(fov_M, 'GM/c^2')
)
obs_params = {
    'mjd': 57851,                       # night of april 6-7, 2017
    'timetype': 'GMST',
    'nt': nt,                           # number of time samples 
    'tstart': tstart.to('hr').value,    # start of observations
    'tstop': tstop.to('hr').value,      # end of observation 
    'tint': 30.0,                       # integration time,
    'array': eh.array.load_txt('../eht_arrays/{}.txt'.format(array))
}
obs_empty = bhnerf.observation.empty_eht_obs(**obs_params)
fov_rad = (fov_M * consts.GM_c2(consts.sgra_mass) / consts.sgra_distance.to('m')) * units.rad
psize = fov_rad.value / geos.alpha.size 
obs_args = {'psize': psize, 'ra': obs_empty.ra, 'dec': obs_empty.dec, 'rf': obs_empty.rf, 'mjd': obs_empty.mjd}
t_frames = np.linspace(tstart, tstop, nt)
image_plane = bhnerf.emission.image_plane_dynamics(emission_0, geos, Omega, t_frames, t_injection)
movie = eh.movie.Movie(image_plane, times=t_frames.value, **obs_args)
obs = bhnerf.observation.observe_same(movie, obs_empty, ttype='direct', seed=None)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


Producing clean visibilities from movie with direct FT . . . 
Applying Jones Matrices to data . . . 
Applying Jones Matrices to data . . . 
Adding thermal noise to data . . . 
Applying a priori calibration with estimated Jones matrices . . . 


In [4]:
"""
Optimize network paremters to recover the 3D emission (as a continuous function) from observations 
Note that logging is done using tensorboardX. To view the tensorboard (from the main directory):
    `tensorboard --logdir runs`
"""
hparams = {'num_iters': 5000, 'lr_init': 1e-4, 'lr_final': 1e-6, 'batchsize': 6}
predictor = bhnerf.network.NeRF_Predictor()
train_pstep = jax.pmap(bhnerf.network.train_step_eht, 
                       axis_name='batch', 
                       in_axes=(0, None, None, 0, 0, 0, 0, None, None, None, None, None, None, None, None, None, None, None), 
                       static_broadcasted_argnums=(1, 2))

# Observation parameters 
chisqdata = eh.imaging.imager_utils.chisqdata_vis
batched_args = bhnerf.optimization.TemporalBatchedArgs.train_step_eht(t_frames, obs, movie.fovx(), movie.xdim, chisqdata)

# Run optimization. Note: rmax constrains the optimization domain to a radius.
rmax = fov_M / 2
current_time = datetime.now().strftime('%Y-%m-%d.%H:%M:%S')
runname = 'tutorial4/recovery.vis.{}'.format(current_time)
state = bhnerf.optimization.run(
    runname, hparams, predictor, train_pstep, geos, Omega, rmax, t_injection, batched_args, 
    emission_true=emission_0, save_period=hparams['num_iters'])

Splitting Observation File into 64 times


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


iteration:   0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
"""
Visualize the recovered 3D emission
This visualization requires ipyvolume: https://ipyvolume.readthedocs.io/en/latest/
"""
import ipyvolume as ipv
emission_estimate = bhnerf.network.sample_3d_grid(state.apply_fn, state.params, float(geos.r.min()), rmax, fov=fov_M)

ipv.figure()
ipv.view(0, -60, distance=2.5)
ipv.volshow(emission_estimate, extent=[(-fov_M/2, fov_M/2)]*3, memorder='F', level=[0, 0.2, 0.7], opacity=[0, 0.2, 0.3], controls=False)
ipv.show()